In [1]:
import os
import tweepy as tw
import pandas as pd
import re
import nltk
from wordcloud import WordCloud,STOPWORDS
nltk.download('punkt')   
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from textblob import TextBlob
import matplotlib.pyplot as plt
# import csv
from datetime import datetime,timedelta ,date

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Teaw\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Teaw\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
consumer_key= 'xxxx'
consumer_secret= 'xxxx'
access_token= 'xxxx'
access_token_secret= 'xxxx'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
class ClawTwitter:
    def __init__(self, hasht:str, langu:str):
        self.listdata = []
        self.hashtag = hasht
        self.lang = langu
        self.data = self.get_datatweet()

    def connect(self):
        try:
            auth = tw.OAuthHandler(consumer_key, consumer_secret)
            auth.set_access_token(access_token, access_token_secret)
            api = tw.API(auth)
            return api
        except:
            print("Error")
            exit(1)

    # def remove_url(self, txt):
    def remove_url(txt):
        """Replace URLs found in a text string with nothing 
        (i.e. it will remove the URL from the string).

        Parameters
        ----------
        txt : string
            A text string that you want to parse and remove urls.

        Returns
        ----------
        The same txt string with url's removed.
        """

        return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

    def remove_url_th(self, txt):
        """Replace URLs found in a text string with nothing 
        (i.e. it will remove the URL from the string).

        Parameters
        ----------
        txt : string
            A text string that you want to parse and remove urls.

        Returns
        ----------
        The same txt string with url's removed.
        """

        return " ".join(re.sub("([^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''|(\w+:\/\/\S+))", "", txt).split())

    # def clean_Text(self, text):
    #     text = text.lower()
    #     text = re.sub(r'(@[A-Za-z0-9_]+)', '', text)
    #     # text = re.sub(r'(\w+:\/\/\S+)','',text)
    #     text = re.sub('http://\S+|https://\S+', '', text)
    #     text = re.sub(r'[^\w\s\n]', '', text)
    #     token_words = word_tokenize(text)
    #     text = [word for word in token_words if not word in stopwords.words()]
    #     text = ' '.join(text)
    #     return text

    def stemmer(self, text):
        porter_text = PorterStemmer()
        token_words = word_tokenize(text)
        stem_word = []
        for word in token_words:
            stem_word.append(porter_text.stem(word))
        return " ".join(stem_word)

    def sentiment(self, text):
        sentiment_text = text.sentiment.polarity
        if  sentiment_text > 0:
            return 'positive'
        elif sentiment_text < 0:
            return 'negative'
        else:
            return 'neutral'

    def get_datatweet(self):
        api = self.connect()
        # today = date.today()
        today = datetime.utcnow().date()
        today_str = today.strftime("%Y-%m-%d")
        tweets = tw.Cursor(api.search_tweets,
                    q=self.hashtag + " -filter:retweets",
                    lang=self.lang,
                    since= today_str).items(50)
                    
        # tweet_set = set()  
        tweet_list = []    
        for tweet in tweets:
            # tweet_set.add(tweet)
            tweet_list.append(tweet)
        tweets = tweet_list
        # print("tweet_set",tweets)   
        # tweets = list(tweet_set)
        # print("tweets",tweets)

        all_tweet_delurl = [ClawTwitter.remove_url(tweet.text) for tweet in tweets]
        data_tweet = [[self.hashtag, tweet.user.screen_name,
                        tweet.user.location if tweet.user.location != '' else 'unknown',
                        tweet.created_at.replace(tzinfo = None) + timedelta(hours = 7),
                        tweet.text,
                        # ClawTwitter.remove_url(tweet.text),
                        tweet.retweet_count,
                        tweet.favorite_count,
                        # self.sentiment(TextBlob(self.stemmer(self.clean_Text(tweet.text)))),
                        self.sentiment(TextBlob(self.stemmer(ClawTwitter.remove_url(tweet.text)))),
                        f"https://twitter.com/twitter/statuses/{tweet.id}"] for tweet in tweets]

        return data_tweet

    def show_data(self, data_tweet:list):
        tweet_frame = pd.DataFrame(data=data_tweet,
                              columns=['topic','user','location','post date','tweet','retweet','likes','sentiment','tweet link'])
        return tweet_frame
        # self.twitter_to_xlsx(tweet_text)

    # def twitter_to_xlsx(self, tweets):
    #     tweets.to_excel("#REDVELVET.xlsx", engine="openpyxl", index=False)

In [4]:
test = ClawTwitter("#REDVELVET", "en")

Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since
Unexpected parameter: since


In [5]:
test.show_data(test.data)

,topic,user,location,post date,tweet,retweet,likes,sentiment,tweet link
0,#REDVELVET,RVStreamingTeam,Red Castle,2022-03-28 12:59:07,Yay! We finally surpassed 20M views on YouTube...,1,4,neutral,https://twitter.com/twitter/statuses/150832277...
1,#REDVELVET,redvelvet_sr,unknown,2022-03-28 12:57:05,Let’s support @RVsmtown 💕\n\nSTREAM ‘Feel My R...,1,0,neutral,https://twitter.com/twitter/statuses/150832226...
2,#REDVELVET,RVStatsBase,unknown,2022-03-28 12:56:58,Here's for prev hour:\n#Feel_My_Rhythm by @RVs...,4,13,neutral,https://twitter.com/twitter/statuses/150832223...
3,#REDVELVET,SeulReneDaily,ReVeluv🎀💖💛💙💚💜,2022-03-28 12:56:22,|OFFICIAL| 220328 #RedVelvet #레드벨벳 Japan 1st F...,0,0,positive,https://twitter.com/twitter/statuses/150832208...
4,#REDVELVET,luvierevelie,unknown,2022-03-28 12:56:15,I think we should also give star donation in I...,1,1,neutral,https://twitter.com/twitter/statuses/150832205...
5,#REDVELVET,chokecherrychuu,Black/she/they,2022-03-28 12:56:14,#RedVelvet Heardle #1\n\n🔊🟩⬜️⬜️⬜️⬜️⬜️\n\nPlay ...,0,0,neutral,https://twitter.com/twitter/statuses/150832204...
6,#REDVELVET,aseuloist,she/they,2022-03-28 12:55:41,beach na peke?!? \n\n#RedVelvet Heardle #2\n\n...,0,0,neutral,https://twitter.com/twitter/statuses/150832191...
7,#REDVELVET,luvie_ly_,unknown,2022-03-28 12:55:40,This is actually so gooooood and THE VOCALSS??...,0,0,neutral,https://twitter.com/twitter/statuses/150832190...
8,#REDVELVET,RVStatsBase,unknown,2022-03-28 12:55:35,#Feel_My_Rhythm by @RVsmtown 220321-220327 K-C...,29,62,neutral,https://twitter.com/twitter/statuses/150832188...
9,#REDVELVET,bjhfart,he/him (asian),2022-03-28 12:53:53,#RedVelvet Heardle #1\n\n🔊🟩⬜️⬜️⬜️⬜️⬜️\n\nPlay ...,0,0,neutral,https://twitter.com/twitter/statuses/150832145...


In [8]:
# keyword_th = ['น้องเค้ก', 'ลัฟวี่', 'จีมิน', 'จองกุก', 'ซึลกิ', 'เวนดี้', 'จอย', 'เยริ',
#         'สวย', 'ตึกชมพู']
keyword_en = ['teaser', 'irene', 'seulgi', 'wendy', 'joy', 'yeri', 'mv', 'music', 'song', 'album',
            'beautiful', 'red', 'velvet', 'out', 'now', 'wildside', 'masterpiece', 'rapper', 'vocal', 'dance']
keyword = ['teaser', 'irene', 'seulgi', 'wendy', 'joy', 'yeri', 'mv', 'music', 'song', 'album',
            'beautiful', 'red', 'velvet', 'out', 'now', 'wildside', 'masterpiece', 'rapper', 'vocal', 'dance']

def count_keyword(hasht):
    # print('here')
    # today = date.today()
    today = datetime.utcnow().date()
    today_str = today.strftime("%Y-%m-%d")
    for i in range(len(keyword)):
        tweets = tw.Cursor(api.search_tweets,
            q=hasht + " -filter:retweets",
            since= today_str).items(20)

    tweet_list = []    
    for tweet in tweets:
        # tweet_list.add(tweet)
        tweet_list.append(tweet)
    tweets = tweet_list
    # print('hi')
    # tweet_set = set()  
    #     # tweet_list = []    
    # for tweet in tweets:
    #     tweet_set.add(tweet)
    #         # tweet_list.append(tweet)
    #     # tweets = tweet_list
    #     # print("tweet_set",tweets)   
    # tweets = list(tweet_set)
    #     # print("tweets",tweets)
    

    key_count = []
    # b = [ClawTwitter.remove_url(tweet.text) for tweet in tweets]
    all_tweet_delurl = [ClawTwitter.remove_url(tweet.text) for tweet in tweets]
    for i in range(len(keyword)):
        # print('hey')
        a = 0
        for j in range(len(all_tweet_delurl)):
            if keyword[i] in all_tweet_delurl[j]:
                a = all_tweet_delurl[j].lower().count(keyword[i].lower())
                a += a
        key_count.append(a)


    return key_count

In [9]:
cw = count_keyword("#REDVELVET")
table = pd.DataFrame(data = {"Topic" : "#REDVELVET", "Keyword" : keyword,
        "Count" : cw})

table

Unexpected parameter: since
Unexpected parameter: since


,Topic,Keyword,Count
0,#REDVELVET,teaser,0
1,#REDVELVET,IRENE,0
2,#REDVELVET,seulgi,0
3,#REDVELVET,wendy,0
4,#REDVELVET,joy,0
5,#REDVELVET,yeri,0
6,#REDVELVET,mv,0
7,#REDVELVET,music,0
8,#REDVELVET,song,0
9,#REDVELVET,album,0
